In [86]:
import os
import re
import fitz
import pandas as pd

# Ruta de la carpeta que contiene los archivos PDF
carpeta_pdf = "C:/Users/monic/OneDrive/Escritorio/hora"

# Inicializar una lista para almacenar los DataFrames de cada archivo
dataframes = []

# Obtener la lista de archivos en la carpeta
archivos_pdf = [archivo for archivo in os.listdir(carpeta_pdf) if archivo.lower().endswith('.pdf')]

# Iterar sobre todos los archivos en la carpeta
for archivo_pdf in archivos_pdf:
    # Construir la ruta completa del archivo
    ruta_completa_pdf = os.path.join(carpeta_pdf, archivo_pdf)

    # Abrir el documento PDF
    documento = fitz.open(ruta_completa_pdf)

    # Crear una cadena vacía para almacenar el texto modificado
    texto_modificado = ""

    # Iterar sobre todas las páginas del documento
    for num_pagina in range(documento.page_count):
        # Acceder a la página
        pagina = documento.load_page(num_pagina)

        # Convertir el contenido de la página a texto
        texto = pagina.get_text("text")

        # Reemplazar tabuladores con "\n"
        texto_modificado += re.sub(r'\t', r'\\n', texto)

    # Reemplazar ": \n" por ":"
    texto_modificado = re.sub(r':\s*\n', r': ', texto_modificado)

    # Inicializar un diccionario para almacenar todos los datos
    datos = {}

    # Filtrar las líneas deseadas
    lineas = texto_modificado.split('\n')
    for linea in lineas:
        if re.match(r'^RFC|^captura|^fecha y hora de presentación|^operación|^Número', linea, re.IGNORECASE):
            # Verificar si la línea contiene el patrón de división
            if ':' in linea:
                # Dividir la línea en clave y valor
                clave, valor = re.split(r':\s*', linea, 1)
                # Convertir la clave a minúsculas
                clave = clave.strip().lower()
                # Agregar a un solo diccionario
                datos[clave] = valor.strip()

    # Convertir el diccionario en un DataFrame de pandas con una sola fila
    df = pd.DataFrame([datos])

    # Agregar el DataFrame a la lista
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
resultado_final = pd.concat(dataframes, ignore_index=True)

# Quitar espacios en la columna "captura"
resultado_final['captura'] = resultado_final['captura'].str.replace(' ', '')

# Dividir la columna "fecha y hora" en dos columnas "fecha" y "hora"
resultado_final[['fecha', 'hora']] = resultado_final['fecha y hora de presentación'].str.split(expand=True)

# Eliminar la columna original "fecha y hora de presentación"
#resultado_final = resultado_final.drop(columns=['fecha y hora de presentación'])

# Imprimir el DataFrame final
#print(resultado_final)

# Guardar el DataFrame final como un archivo Excel
resultado_final.to_excel('datos_excel_final.xlsx', index=False)


    número de operación            rfc fecha y hora de presentación  \
0             557263677  AACM681228AQ4             29/11/2023 10:33   
1          230241587029  AACM681228AQ4             29/11/2023 10:36   
2             557190947  AAHF740327MH9             28/11/2023 17:09   
3             557076522  AAOD890817RQ5             28/11/2023 09:49   
4             557001664   AIC121112UJA             27/11/2023 16:44   
..                  ...            ...                          ...   
293           556892204   WCF210324T95             27/11/2023 09:40   
294        230771599563   YVE960411LV1             17/11/2023 10:39   
295           555595832   YVE960411LV1             17/11/2023 11:59   
296           557142479   ZPE150508JX9             28/11/2023 13:39   
297        230730157229   ZPE150508JX9             28/11/2023 13:57   

                  captura     operación       fecha   hora  
0    042378PM190040675236  123353280450  29/11/2023  10:33  
1    042378PZ150040670208